Alaska updates daily at ??

Alaska doesn't offer county/borough level data.

In [1]:
from selenium import webdriver
import time
import pandas as pd
import pendulum
import re
import yaml
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
#chrome_options.add_argument("--disable-extensions")
#chrome_options.add_argument("--disable-gpu")
#chrome_options.add_argument("--no-sandbox) # linux only
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")
chrome_options.add_argument("user-agent=[Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:73.0) Gecko/20100101 Firefox/73.0]")

In [2]:
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f.read())

In [3]:
state = 'AK'

In [4]:
scrape_timestamp = pendulum.now().strftime('%Y%m%d%H%M%S')

In [5]:
url = 'http://dhss.alaska.gov/dph/Epi/id/Pages/COVID-19/monitoring.aspx'

In [3]:
def fetch():
    driver = webdriver.Chrome('../20190611 - Parts recommendation/chromedriver', options=chrome_options)

    driver.get(url)
    time.sleep(5)
    datatbl = driver.find_element_by_css_selector('table')

#     data = []
#     for row in datatbl.find_elements_by_css_selector('tr'):
#         row_data = [row.find_element_by_css_selector('th').text]
#         data.append(row_data + [cell.text for cell in row.find_elements_by_css_selector('td')])
    
    data = []
    for row in datatbl.find_elements_by_css_selector('tr'):
        data.append([cell.text for cell in row.find_elements_by_css_selector('td')])
        
    page_source = driver.page_source
    driver.close()
    return pd.DataFrame(data, columns=['county','travel_related','non_travel_related','positive_cases']), page_source

In [ ]:
def save(df, source):
    df.to_csv(f"{config['data_folder']}/{state}_county_{scrape_timestamp}.txt", sep='|', index=False)

    with open(f"{config['data_source_backup_folder']}/{state}_county_{scrape_timestamp}.html", 'w') as f:
        f.write(source)

In [ ]:
def run():
    df, source = fetch()
    save(df, source)